<a href="https://www.kaggle.com/code/tanvikiran27/harmonic-invariant-basis?scriptVersionId=246170022" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import itertools as it
import sympy as sp 
import pandas as pd
from sympy import symbols, Function, diff, FiniteSet, simplify, re
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC

import os
import zipfile
import shutil

for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/degeneracy-tables/n=4.csv
/kaggle/input/degeneracy-tables/n=3.csv
/kaggle/input/degeneracy-tables/n=2.csv
/kaggle/input/degeneracy-tables/dataset-metadata.json


In [2]:
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
from kaggle_secrets import UserSecretsClient
if os.path.exists('/root/.kaggle/'):
    pass
else:
    os.mkdir('/root/.kaggle/')
kaggle_API_key = UserSecretsClient().get_secret("key")

with open('/root/.kaggle/kaggle.json', 'w') as fid:
    fid.writelines(f'{{"username":"tanvikiran27","key":"{kaggle_API_key}"}}')

!chmod 600 /root/.kaggle/kaggle.json
print('Yay, it worked!')

Yay, it worked!


In [3]:
# https://www.kaggle.com/code/abdurrakibmollah/move-files-from-input-to-output-folder
dst_path = "/kaggle/working/"
for dirname, _, filenames in os.walk('/kaggle/input/degeneracy-tables'):
     for filename in filenames:
        src_path = os.path.join(dirname, filename)
        shutil.copy(src_path, dst_path)
        print('Copied '+filename)

Copied n=4.csv
Copied n=3.csv
Copied n=2.csv
Copied dataset-metadata.json


$n$ = dimension ($\mathbb{R}^n$ or $\mathbb{C}^n$) \
$k$ = real degree \
$(p,q)$ = complex bidegree

In [4]:
####### Change constants here #######
n = 3
k = 3
p = 2
q = 1
#####################################

# Setting up symbols
f = symbols('f', cls = Function)
x,y,z,w,zeta = symbols('x y z w zeta')
zs = symbols(f'z1:{n+1}', complex=True)
zbar = symbols(f'zbar1:{n+1}', complex=True)
ls = symbols(f'l1:{n+1}', int=True)

# newton potential (R^n) 
mag_real = 0
for i in range(n):
    mag_real += zs[i]**2 ## Euclidean norm squared
np_real = sp.sqrt(mag_real)**(2-n)

# newton potential (C^n)
mag_complex = 0
for i in range(n):
    mag_complex += zs[i]*zbar[i] ## Complex modulus squared
np_complex = sp.sqrt(mag_complex)**(2-2*n)

In [5]:
from sympy.matrices import diag

# Cyclic group (roots of unity)
m=5
zeta = exp(2*pi*I/m)
g = diag(zeta, zeta**(-1))
G = FiniteSet()
for i in range(m):
    G += FiniteSet(g**i)
G

{Matrix([
[1, 0],
[0, 1]]), Matrix([
[exp(-4*I*pi/5),             0],
[             0, exp(4*I*pi/5)]]), Matrix([
[exp(-2*I*pi/5),             0],
[             0, exp(2*I*pi/5)]]), Matrix([
[exp(2*I*pi/5),              0],
[            0, exp(-2*I*pi/5)]]), Matrix([
[exp(4*I*pi/5),              0],
[            0, exp(-4*I*pi/5)]])}

In [6]:
# Multi index differentiation
def D(f,alpha,zs):
    df = f
    for i in range(n):
        df = diff(df,(zs[i],alpha[i]))
    return df

In [7]:
# Returns valid n-tuples
# https://www.geeksforgeeks.org/python-program-to-convert-dictionary-to-list-by-repeating-keys-corresponding-value-times/
def get_alpha_beta(deg):
    alphas = FiniteSet()
    for part in partitions(deg, m=n):
        part = list(it.chain.from_iterable(it.repeat(k, v) for k, v in part.items()))
        while len(part) < n:
            part.append(0)
        for perm in it.permutations(part):
            alphas += FiniteSet(perm)
    return alphas

In [8]:
# Turns rational functions in our basis into polynomials
def make_polynomial(f, mag):
    f = sp.numer(f).subs(mag, x)
    while f.is_polynomial(zs,zbar,x) == False:
        f = simplify(f * x)
        f = sp.numer(f).subs(mag, x)
    return expand(f.subs(x,abs(z)**2))

In [9]:
# For H_k(R^n) *NOT C^n
def real_dim():
    return nC(n+k-1,k) - nC(n+k-3,k-2)

In [10]:
# For H_k(C^n)
def dim_H_k():
    return nC(2*n+k-1,k) - nC(2*n+k-3,k-2)

In [11]:
# For H_p,q(C^n)
def dim_H_pq():
    return nC(n+p-1,p)*nC(n+q-1,q) - nC(n+p-2,p-1)*nC(n+q-2,q-1)

In [12]:
# Finding basis for H_k(R^n) - see Axler's Harmonic Function Theory p. 92
def real_basis():
    basis = FiniteSet()
    alphas = get_alpha_beta(k)
    for a in alphas:
        if a[0] > 1:
            alphas -= FiniteSet(a)
    print(f'# of elements = {len(alphas)}')
    print(f'Dimension from combinatorial expression = {real_dim()} \n')
    for a in alphas:
        d = D(np_real, a, zs)
        d = make_polynomial(d, mag_real)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [13]:
# Finding basis for H_p,q(C^n)
def basis_H_pq():
    basis = FiniteSet()
    alphas = get_alpha_beta(p)
    betas = get_alpha_beta(q)
    ab = FiniteSet()
    for a in alphas:
        for b in betas:
            if a[0]==0 or b[0]==0:
                ab += FiniteSet((a,b))
    print(f'# of elements = {len(ab)}')
    print(f'Dimension from combinatorial expression = {dim_H_pq()} \n')
    for (a,b) in ab:
        d = D(np_complex,b,zs)
        d = D(d,a,zbar)
        d = make_polynomial(d, mag_complex)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [14]:
# Regular complex Laplacian
def box(f):
    laplacian = 0
    for i in range(n):
        laplacian += diff(diff(f,zbar[i]),zs[i])
    return laplacian

In [15]:
# Kohn Laplacian as given in 2018 REU paper
def box_b(f):
    box_b = 0
    # first term
    for i in range(n): 
        box_b += -2*diff(diff(f,zbar[i]), zs[i])
        
    # second term
    for i in range(n): 
        box_b += 2*diff(zs[i]*q*f, zs[i])
        
    # third term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += 2*diff(zs[a]*zbar[i]*diff(f,zbar[i]), zs[a])
            
    # fourth term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += -2*diff(zs[a]*zbar[i]*zs[i]*q*f, zs[a])
    return box_b

In [16]:
# Just in case :)
def is_harmonic(f):
    return (box_b(f)/f).simplify().is_polynomial(mag_complex)

In [17]:
# Returns the Ikeda generating function for a given lens space
def F(k,ls):
    F = 0
    zeta = exp(2*pi*I/k)
    for m in range(k):
        frac = 1-z*w
        for i in range(len(ls)):
            frac /= (z-zeta**(-m*ls[i]))*(w-zeta**(m*ls[i]))
        F += frac
    F = (1/k)*F
    return F

In [18]:
# Coefficients from Taylor series expansion for a two-variable function- read more at
# https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
def taylor_coeffs(f,deg):
    table = []
    for i in range(deg+1):
        row = []
        for j in range(deg+1-i):
            d = diff(f,w,j)
            d = diff(d,z,i)
            d = lambdify([z,w],d)
            term = re(d(0,0)/(factorial(i)*factorial(j))).round()
            row.append(term)
        while len(row) < deg+1:
            row.append('-')
        table.append(row)
    return table

In [19]:
# Gets a single degeneracy table (as a list of arrays) from the dataset
def clean_col(df,col):
    table = df[col].to_list()
    num_table = []
    while type(table[-1]) is float:
        table = table[:-1]
    for row in table:
        num_table.append(eval(row))
    return num_table

In [20]:
# Turns the order k and a list of l values into the name of a lens space (string)
# To avoid duplicates, the l values are sorted in ascending order (their order doesn't 
# affect the Ikeda generating function and thus doesn't affect the degeneracy tables)
def make_lens_str(k,ls):
    lens = "L("+str(k)+";"
    ls.sort()
    for l in ls:
        lens += str(l)+","
    lens = lens[:-1]
    lens += ")"
    return lens

In [21]:
# File lookup
def find_file(file):
    for dirname, _, filenames in os.walk('/kaggle/working'):
        if file in filenames:
            return True
    return False

In [22]:
# Update dataset
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
def update_data():
    with open('/kaggle/working/dataset-metadata.json', 'w') as json_fid:
        json_fid.write('{\n  "title": "Degeneracy Tables",\n  "id": "tanvikiran27/degeneracy-tables",\n  "licenses": [{"name": "CC0-1.0"}]}')
    !kaggle datasets version -m new

In [23]:
# In case you need to clear the dataset for some reason
# CAUTION: This will delete the .csvs from the output directory and can potentially
# delete everything in the dataset- use with caution...
# https://www.kaggle.com/code/focusedmonk/delete-output-files-in-kaggle
def clear_csvs():
    import pathlib
    files_to_delete = './*.csv' # this considers only ".csv" files. If you want to delete all files, use "./*."
    files_list = pathlib.Path(os.getcwd()).glob(files_to_delete)
    for file_path in files_list:
        os.remove(file_path)

In [24]:
# Creates a degeneracy table for a lens space with all 0 <= p,q <= deg
# If there's already an entry for the lens space, this function finds it in the dataset
# Otherwise it adds the new table to the dataset
def degeneracy_table(k,ls,deg):
    N = len(ls)
    lens = make_lens_str(k,ls)
    file = 'n=' + str(N) + '.csv'
    path = "/kaggle/working/" + file
    if find_file(file):
        try:
            data = pd.read_csv(path)
        except pd.errors.EmptyDataError:
            data = pd.DataFrame()
    else:
        data = pd.DataFrame()
    if lens not in data or deg >= len(clean_col(data,lens)):
        f = F(k,ls)
        T = taylor_coeffs(f,deg)
        if lens in data:
            data = data.drop(columns=[lens])
        new = pd.DataFrame({lens:T})
        added = data.merge(new, left_index = True, right_index = True, how = 'outer')
        added.to_csv(path, index = False)
        print("Created/updated " + file)
        update_data()
    else:
        T = clean_col(data,lens)
    return pd.DataFrame(T)

In [50]:
degeneracy_table(7,[-3,-2,2,3],6)

,0,1,2,3,4,5
0,1,0,2,2,5,8
1,0,3,4,10,18,-
2,2,4,12,24,-,-
3,2,10,24,-,-,-
4,5,18,-,-,-,-
5,8,-,-,-,-,-


In [40]:
degeneracy_table(7,[-5,-2,2,5],5)

Created/updated n=4.csv
Starting upload for file n=4.csv
100%|██████████████████████████████████████| 1.23k/1.23k [00:00<00:00, 3.77kB/s]
Upload successful: n=4.csv (1KB)
Skipping folder: .virtual_documents; use '--dir-mode' to upload folders
Starting upload for file n=2.csv
100%|██████████████████████████████████████| 2.95k/2.95k [00:00<00:00, 8.97kB/s]
Upload successful: n=2.csv (3KB)
Starting upload for file n=3.csv
100%|██████████████████████████████████████████| 953/953 [00:00<00:00, 2.89kB/s]
Upload successful: n=3.csv (953B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5
0,1,0,4,0,9,0
1,0,7,0,20,0,-
2,4,0,26,0,-,-
3,0,20,0,-,-,-
4,9,0,-,-,-,-
5,0,-,-,-,-,-


In [48]:
degeneracy_table(7,[-5,-2,2,5],9)

Created/updated n=4.csv
Starting upload for file n=4.csv
100%|██████████████████████████████████████| 1.55k/1.55k [00:00<00:00, 4.22kB/s]
Upload successful: n=4.csv (2KB)
Skipping folder: .virtual_documents; use '--dir-mode' to upload folders
Starting upload for file n=2.csv
100%|██████████████████████████████████████| 3.30k/3.30k [00:00<00:00, 8.67kB/s]
Upload successful: n=2.csv (3KB)
Starting upload for file n=3.csv
100%|██████████████████████████████████████████| 953/953 [00:00<00:00, 2.84kB/s]
Upload successful: n=3.csv (953B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6,7,8,9
0,1,0,4,0,9,0,16,16,25,36
1,0,7,0,20,0,39,28,64,84,-
2,4,0,26,0,60,36,106,120,-,-
3,0,20,0,70,40,140,144,-,-,-
4,9,0,60,40,155,156,-,-,-,-
5,0,39,36,140,156,-,-,-,-,-
6,16,28,106,144,-,-,-,-,-,-
7,16,64,120,-,-,-,-,-,-,-
8,25,84,-,-,-,-,-,-,-,-
9,36,-,-,-,-,-,-,-,-,-


In [46]:
degeneracy_table(11,[-1,1],8)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0,1,0,1,0,1,0,1,0,1,2,1
1,0,1,0,1,0,1,0,1,0,1,2,1,-
2,1,0,1,0,1,0,1,0,1,2,1,-,-
3,0,1,0,1,0,1,0,1,2,1,-,-,-
4,1,0,1,0,1,0,1,2,1,-,-,-,-
5,0,1,0,1,0,1,2,1,-,-,-,-,-
6,1,0,1,0,1,2,1,-,-,-,-,-,-
7,0,1,0,1,2,1,-,-,-,-,-,-,-
8,1,0,1,2,1,-,-,-,-,-,-,-,-
9,0,1,2,1,-,-,-,-,-,-,-,-,-


In [51]:
degeneracy_table(1,[0,0],6)

Created/updated n=2.csv
Starting upload for file n=4.csv
100%|██████████████████████████████████████| 1.55k/1.55k [00:00<00:00, 4.67kB/s]
Upload successful: n=4.csv (2KB)
Skipping folder: .virtual_documents; use '--dir-mode' to upload folders
Starting upload for file n=2.csv
100%|██████████████████████████████████████| 3.52k/3.52k [00:00<00:00, 10.6kB/s]
Upload successful: n=2.csv (4KB)
Starting upload for file n=3.csv
100%|██████████████████████████████████████████| 953/953 [00:00<00:00, 2.88kB/s]
Upload successful: n=3.csv (953B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6
0,1,2,3,4,5,6,7
1,2,3,4,5,6,7,-
2,3,4,5,6,7,-,-
3,4,5,6,7,-,-,-
4,5,6,7,-,-,-,-
5,6,7,-,-,-,-,-
6,7,-,-,-,-,-,-
